In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np
import os

import ICARUS.Visualization as vis
from ICARUS.Database.Database_3D import Database_3D

In [ ]:
HOMEDIR = os.getcwd()

# Open Database

In [ ]:
db = Database_3D(HOMEDIR)


### Select Plane

In [ ]:
planenames = db.getPlanes()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}")
# index = int(input("Select a plane: "))
# planenames = [planenames[index]]

plane = db.Planes[planenames[1]]
planenames = [planenames[1]]
# dynplane = db.dynPlanes[planenames[0]]

# Add XFLR Polars

In [ ]:
for name in planenames:
    try:
        db.importXFLRpolar(f"{HOMEDIR}/ICARUS/XFLR5/{name}.txt", name)
        print(f"Imported XFLR polar for {name}")
        planenames.append(f"XFLR_{name}")
    except FileNotFoundError:
        print(f"No XFLR polar found for {name}")
        # db.importXFLRpolar(f"{HOMEDIR}/ICARUS/XFLR5/polar.txt")

# Airplane Visualization

In [ ]:
plane.visAirplane()

# Polars

In [ ]:
from ICARUS.Visualization.airplanePolars import plotAirplanePolars

In [ ]:
db.makeData()
plotAirplanePolars(db.Data,planenames,["2D"],size=(10,10))

# Case Convergence

In [ ]:
from ICARUS.Visualization.convergence import plotConvergence

In [ ]:
angles2show = [-6.,-3., 0.]
plotConvergence(db.Convergence, plane.name, ['All'],
                solvers= ["2D"],plotError = False,size=(16,7))

# Sensitivity Plot

In [ ]:
from ICARUS.Visualization.sensitivity import plotSensitivity

In [ ]:
# pertr = dynplane.pertubResults
# print(dynplane.trim)
# trim = pertr[pertr['Type'] == 'Trim']
# dynplane.sensResults['u'].sort_values(by=['Epsilon'])
# plotSensitivity(dynplane.sensResults,dynplane,trim,relative = 0,vars2s=['r'])

# Wake Visualization

In [ ]:
from ICARUS.Visualization.GNVPwake import GNVPwake
from ICARUS.Database.Database_3D  import ang2case

In [ ]:
CASE = ang2case(-6.)
GNVPwake(db.Planes[planenames[0]],CASE)

# Loads Visualization

In [ ]:
from ICARUS.Visualization.GNVPstrips import GNVPstrips

stripDATA = GNVPstrips(db.Planes[planenames[0]],CASE,HOMEDIR)
stripDATA